- This Notebook pulls information about the weather by City

In [4]:
!pip3 install datetime

In [1]:
# Import your module or file
from    IPython.core.display import JSON
from    dotenv        import load_dotenv
from    datetime import datetime     
import  pandas               as pd
import  geocoder, requests, json, os, pytz

ModuleNotFoundError: No module named 'dotenv'

In [3]:
load_dotenv()

GEO_USERNAME    = os.environ.get('GEO_USERNAME')
WEATHER_API_KEY = os.environ.get('WEATHER_API_KEY')

In [4]:
#wc = pd.read_csv('/Users/jlma/Documents/Curso de Data Science/5-Data Engineer/Project3/Data-Engineering-Pipelines-on-the-Cloud/world_cities.csv')

In [5]:
#@title Country Selection { run: "auto" }
#Country = "Venezuela" #@param ["Germany", "Venezuela","Spain","Andorra"]
#cities_df = list(wc.set_index('country').loc[Country].name)
#cities_df

In [6]:
eu = pd.read_csv('./EuroCitiesPopulation13.csv')

In [7]:
eu_cities = list(eu.set_index('country').name)

In [8]:
eu_cities

['London',
 'Berlin',
 'Madrid',
 'Paris',
 'Bucharest',
 'Budapest',
 'Hamburg',
 'Warsaw',
 'Vienna',
 'Barcelona',
 'Stockholm',
 'Belgrade',
 'Munich']

 - The next code is to get information from geonames

In [9]:
g = geocoder.geonames('berlin', key=GEO_USERNAME)
g.geojson#['features'][0]['properties']['country_code']

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'address': 'Berlin',
    'class_description': 'city, village,...',
    'code': 'PPLC',
    'country': 'Germany',
    'country_code': 'DE',
    'description': 'capital of a political entity',
    'feature_class': 'P',
    'geonames_id': 2950159,
    'lat': '52.52437',
    'lng': '13.41053',
    'ok': True,
    'population': 3426354,
    'raw': {'adminCode1': '16',
     'lng': '13.41053',
     'geonameId': 2950159,
     'toponymName': 'Berlin',
     'countryId': '2921044',
     'fcl': 'P',
     'population': 3426354,
     'countryCode': 'DE',
     'name': 'Berlin',
     'fclName': 'city, village,...',
     'adminCodes1': {'ISO3166_2': 'BE'},
     'countryName': 'Germany',
     'fcodeName': 'capital of a political entity',
     'adminName1': 'Berlin',
     'lat': '52.52437',
     'fcode': 'PPLC'},
    'state': 'Berlin',
    'state_code': '16',
    'status': 'OK'},
   'geometry': {'type': 'Point', 'coordinate

In [18]:
weather_df = pd.DataFrame(columns=['city','time_utc','local_time','temperature','humidity','cloudiness_pc','wind_speed','precipitation_prob','rain_volume','snow_volume'])
airpotrs = pd.DataFrame(columns=['city','a_lat','a_lon','icao','iata','name','country_code'])

In [ ]:
lat=[]
lng=[]
population=[]
country_code=[]
time_zone=[]


for city in eu_cities:
    g = geocoder.geonames(city, key=GEO_USERNAME)
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={WEATHER_API_KEY}&units=metric"
    lat             = lat+[(g.geojson['features'][0]['properties']['lat'])]
    lng             = lng+[(g.geojson['features'][0]['properties']['lng'])]
    population      = population+[(g.geojson['features'][0]['properties']['population'])]
    country_code    = country_code+[(g.geojson['features'][0]['properties']['country_code'])]
    
    weather_js      = requests.get(url).json()
    time_zone       = time_zone+[(weather_js["city"]["timezone"])]

    for x in weather_js["list"]:
            try:
                rain_vol = x["rain"]["3h"]
            except KeyError as e1:
                #print('I got a KeyError - reason "%s"' % str(e1))
                rain_vol = 0
                
            try:
                snow_vol = x["snow"]["3h"]
            except KeyError as e2:
                #print('I got a KeyError - reason "%s"' % str(e2))
                snow_vol = 0
                
            weather_df=weather_df.append({'city':city,
                        'time_utc':x["dt"],
                        'local_time':datetime
                                    .fromtimestamp(x["dt"]+weather_js["city"]["timezone"])
                                    .strftime("%B %d, %Y %I:%M:%S"),
                        'temperature':x["main"]["temp"],
                        'humidity':x["main"]["humidity"],
                        'cloudiness_pc':x["clouds"]["all"],
                        'wind_speed':x["wind"]["speed"],
                        'precipitation_prob':x["pop"],
                        'rain_volume':rain_vol,#x["rain"]["3h"],
                        'snow_volume':snow_vol#x["snow"]["3h"]
                        }
                        ,ignore_index=True)
    
    lat = g.geojson['features'][0]['properties']['lat']
    lon = g.geojson['features'][0]['properties']['lng']
    
    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat}/{lon}/km/50/16"
    
    querystring = {"withFlightInfoOnly":"0"}
    
    headers = {
            	"X-RapidAPI-Key": "2ef471dfcbmshec01fd033aae0e3p10a740jsn9a6d81f2bead",
            	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
    
    response = requests.request("GET", url, headers=headers)#, params=querystring)
    print('Status code',response.status_code)
    airp_js = response.json()
    
    for a in airp_js["items"]:
        airpotrs = airpotrs.append({'city':city,
                      'a_lat':a["location"]["lat"],
                      'a_lon':a["location"]["lon"],
                      'icao':a["icao"],
                      'iata':a["iata"],
                      'name':a["name"]
                      }
                      ,ignore_index=True)
        
flights = pd.DataFrame(columns=['icao','date','num_of_arriv','num_of_depart'])

n_days=2
for a_icao in airpotrs['icao']:
    n_arr = 0
    n_dep = 0
    for j in range(n_days):
        date = (datetime.date.today() + datetime.timedelta(days=j)).strftime('%Y-%m-%d') 
        t1=['00:00','12:00']
        t2=['11:59','23:59']
        
        headers = {
        	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
        	"X-RapidAPI-Key": "2ef471dfcbmshec01fd033aae0e3p10a740jsn9a6d81f2bead"
        }
        
        for i in range(2):
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{a_icao}/{date}T{t1[i]}/{date}T{t2[i]}"
            
            response = requests.request("GET", url, headers=headers)#, params=querystring)
            if response.status_code==200:
                flights_js = response.json()
                n_arr = n_arr + len(flights_js['arrivals'])
                n_dep = n_dep + len(flights_js['departures'])
            else:
                n_arr = np.nan
                n_dep = np.nan
 
        flights = flights.append({'icao':a_icao,
                      'date':date,
                      'num_of_arriv':n_arr,
                      'num_of_depart':n_dep
                      }
                      ,ignore_index=True)

In [ ]:
cities_dic = {'City': eu_cities, 'Country': country_code, 'Population': population, 'Latitud': lat, 'Logitud': lng, 'Time_Zone': time_zone  }
cities_df  = pd.DataFrame(cities_dic)

In [ ]:
cities_df

,City,Country,Population,Latitud,Logitud,Time_Zone
0,London,GB,8961989,51.50853,-0.12574,3600
1,Berlin,DE,3426354,52.52437,13.41053,7200
2,Madrid,ES,3255944,40.4165,-3.70256,7200
3,Rome,IT,2318895,41.89193,12.51133,7200
4,Paris,FR,2138551,48.85341,2.3488,7200
5,Bucharest,RO,1877155,44.43225,26.10626,10800
6,Budapest,HU,1741041,47.49835,19.04045,7200
7,Hamburg,DE,1739117,53.55073,9.99302,7200
8,Warsaw,PL,1702139,52.22977,21.01178,7200
9,Vienna,AT,1691468,48.20849,16.37208,7200




*   The next code is to get information from weather





In [ ]:
weather_df.head(45)

,city,time_utc,local_time,temperature,humidity,cloudiness_pc,wind_speed,precipitation_prob,rain_volume,snow_volume
0,London,1654873200,"June 10, 2022 06:00:00",22.72,49,27,6.11,0,0,0
1,London,1654884000,"June 10, 2022 09:00:00",20.96,51,13,5.58,0,0,0
2,London,1654894800,"June 11, 2022 12:00:00",15.6,70,5,4.11,0,0,0
3,London,1654905600,"June 11, 2022 03:00:00",13.15,81,8,3.26,0,0,0
4,London,1654916400,"June 11, 2022 06:00:00",11.68,84,0,3.04,0,0,0
5,London,1654927200,"June 11, 2022 09:00:00",13.36,76,2,4.28,0,0,0
6,London,1654938000,"June 11, 2022 12:00:00",18.32,53,3,4.96,0,0,0
7,London,1654948800,"June 11, 2022 03:00:00",21.87,40,7,5.63,0,0,0
8,London,1654959600,"June 11, 2022 06:00:00",22,40,10,6.25,0,0,0
9,London,1654970400,"June 11, 2022 09:00:00",19.81,53,25,4.7,0,0,0
